In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import bemcs

%config InlineBackend.figure_format = "retina"

# Example 1: Fault at right angles
Consider a 3-element fault that meets only at right angles, and we impose slip on the node centers. In the following sections we consider 2 related cases. But first, let's look at the geometry of the fault.

In [ ]:
# Geometry of three element fault at right angles
x1 = np.array([-1.0, 0.0, 0.0])
x2 = np.array([0.0, 0.0, 1.0])
y1 = np.array([0.5, 0.5, -0.5])
y2 = np.array([0.5, -0.5, -0.5])

elements = []
element = {}
for i in range(x1.size):
    element["x1"] = x1[i]
    element["y1"] = y1[i]
    element["x2"] = x2[i]
    element["y2"] = y2[i]
    elements.append(element.copy())
elements = bemcs.standardize_elements(elements)
n_elements = len(elements)

plt.figure(figsize=(6, 4))
bemcs.plot_element_geometry(elements)


# Imposed slip (specified in $x,y$ direction)

In [ ]:
# Slip imposed at central nodes of each patch (these are in x,y coordinates)
slip_vector_x = np.array([1.0, 0.0, 1.0])
slip_vector_y = np.array([0.0, -1.0, 0.0])

# plot slip function
plt.figure(figsize=(6, 4))
plt.plot((x1 + x2) / 2, slip_vector_x, "o-", label="$s_x$")
plt.plot((x1 + x2) / 2, slip_vector_y, "o-", label="$s_y$")
plt.xlabel("x")
plt.ylabel("imposed slip")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Design matrices (in x, y coordinates) for slip and slip gradients at each 3qn
stride = 6
max_continuity_row = (n_elements * stride) - stride
matrix_slip, matrix_slip_gradient = bemcs.get_designmatrix_xy_3qn(elements)
matrix_system = np.zeros((n_elements * stride, n_elements * stride))

# Set boundary conditions
# Slip -or- slip gradients at the boundaries (we set them to 0 here)
bc_vector = np.zeros((n_elements * stride))
bc_vector[0:2] = 0.0
bc_vector[-2:] = 0.0
bc_vector[2::stride] = slip_vector_x
bc_vector[3::stride] = slip_vector_y

# Linear operator for BCs as a matrix
# Boundary nodes
matrix_system[0:2, :] = matrix_slip[0:2]
matrix_system[-2:, :] = matrix_slip[-2:, :]

# Patch center nodes
matrix_system[2::stride, :] = matrix_slip[2::stride, :]
matrix_system[3::stride, :] = matrix_slip[3::stride, :]

# Overlapping interior nodes (apply slip continuity and smoothness)
# Slip continuity (x component)
matrix_system[4:-7:6, :] = matrix_slip[4:-7:6, :] - matrix_slip[6:-5:6, :]

# Slip continuity (x component)
matrix_system[5:-6:6, :] = matrix_slip[5:-6:6, :] - matrix_slip[7:-4:6, :]

# Smoothness
matrix_system[6:-5:6, :] = (
    matrix_slip_gradient[4:-7:6, :] - matrix_slip_gradient[6:-5:6, :]
)  # x component
matrix_system[7:-4:6, :] = (
    matrix_slip_gradient[5:-6:6, :] - matrix_slip_gradient[7:-4:6, :]
)  # y component

# Solve for quadratic node coefficients (in local (s,n) coordinates)
quadratic_coefs = np.linalg.inv(matrix_system) @ bc_vector

# Extract (s,n) components and store them in 2 separate vectors
quadratic_coefs_s = np.zeros((len(elements) * 3))
quadratic_coefs_n = np.zeros((len(elements) * 3))
for i in range(len(elements)):
    quadratic_coefs_s[3 * i : 3 * (i + 1)] = quadratic_coefs[6 * i : 6 * i + 3]
    quadratic_coefs_n[3 * i : 3 * (i + 1)] = quadratic_coefs[6 * i + 3 : 6 * (i + 1)]

# Compute displacement, stress kernels 

In [ ]:
# Compute displacement and stress kernels
# Elastic parameters
mu = 1
nu = 0.25

# Observation coordinates
n_obs = 100
width = 2
x_obs = np.linspace(-width, width, n_obs)
y_obs = np.linspace(-width / 2, width / 2, n_obs)
x_obs, y_obs = np.meshgrid(x_obs, y_obs)
x_obs = x_obs.flatten()
y_obs = y_obs.flatten()

# Compute shear and tensile kernels
# kernels are [Kxx, Kyy, Kxy, Gx, Gy] x [Nobs x Ncoefficients]
# the last argument passed is a flag for shear/tensile source (1 - shear, 0 - tensile)
kernels_s = bemcs.get_quadratic_displacement_stress_kernel(
    x_obs, y_obs, elements, mu, nu, 1
)
kernels_n = bemcs.get_quadratic_displacement_stress_kernel(
    x_obs, y_obs, elements, mu, nu, 0
)

# Get displacements and stresses
sxx = kernels_s[0] @ quadratic_coefs_s + kernels_n[0] @ quadratic_coefs_n
syy = kernels_s[1] @ quadratic_coefs_s + kernels_n[1] @ quadratic_coefs_n
sxy = kernels_s[2] @ quadratic_coefs_s + kernels_n[2] @ quadratic_coefs_n
ux = kernels_s[3] @ quadratic_coefs_s + kernels_n[3] @ quadratic_coefs_n
uy = kernels_s[4] @ quadratic_coefs_s + kernels_n[4] @ quadratic_coefs_n

## Plot displacements and stresses for 1a and 1b

In [ ]:
n_skip_plot = 21  # plotting for vectors
bemcs.plot_displacements_stresses(
    elements, n_obs, ux, uy, sxx, syy, sxy, x_obs, y_obs, n_skip_plot
)

# Example 2: Sinusoidal fault

For a sinusoidal fault in an $x,y$ coordinate system, $y = A\sin\left(\frac{2n\pi}{L}\right)$ where $A$ is the amplitude, $n$ is the number of cycles of the sinusoid and $L$ is the fault length in the $x$ dimension. 

In this example we force the slip vector to have purely shear sense slip at the element centers while at the edge nodes continuity and smoothness conditions take over. The resulting slip distribution is rather intriguing! Tensile slip remains 0 at patch centers, as we prescribed, but it is discontinuous and non-zero at overlapping nodes while the slip vector in an $x,y$ coordinate system remains continuous and smooth.

In [ ]:
# flag for slip to be purely shear
slip_purely_shear = True


If you want to test the other option i.e., impose slip in a uniform $x,y$ direction, simply set the variable `slip_purely_shear = False`

In [ ]:
# use a non-uniform mesh
n_elements = 12
L_fault = 1.5  # fault length in x

# specify amplitude and number of wavelengths for sinusoidal fault geometry
y_amplitude, lambda_cycles = 0.3, 1.0

# sinusoidal y-positions for fault elements
sorted_xlocs = np.linspace(
    -L_fault / 2 + L_fault / n_elements,
    L_fault / 2 - L_fault / n_elements,
    n_elements - 1,
)

# rough fault
x1 = np.concatenate((np.array([-L_fault / 2.0]), sorted_xlocs))
x2 = np.concatenate((sorted_xlocs, np.array([L_fault / 2.0])))
y1 = y_amplitude * np.sin(lambda_cycles * 2 * np.pi * x1 / L_fault)
y2 = y_amplitude * np.sin(lambda_cycles * 2 * np.pi * x2 / L_fault)

elements = []
element = {}
for i in range(n_elements):
    element["x1"] = x1[i]
    element["y1"] = y1[i]
    element["x2"] = x2[i]
    element["y2"] = y2[i]
    elements.append(element.copy())
elements = bemcs.standardize_elements(elements)

# slip imposed at central nodes of each patch (these are in x,y coordinates)
slip_values = np.exp(-np.abs(np.linspace(-1, 1, n_elements)) / 2 / (0.5**2))
slip_vector_x = np.zeros_like(slip_values)
slip_vector_y = np.zeros_like(slip_values)
slip_vector_s = np.zeros_like(slip_values)
slip_vector_n = np.zeros_like(slip_values)

# force slip vector to be purely shear (s_n = 0)
if slip_purely_shear == True:
    slip_vector_s = slip_values
    slip_vector_n = np.zeros_like(slip_vector_s)
    slip_vector = np.vstack((slip_vector_s, slip_vector_n)).T
    print("Forcing slip vector at patch centers to be purely shear")
    for i in range(len(elements)):
        slip_vector_x[i], slip_vector_y[i] = (
            slip_vector[i, :] @ elements[i]["inverse_rotation_matrix"]
        )
else:
    slip_vector_x = slip_values
    slip_vector_y = np.zeros_like(slip_vector_x)
    slip_vector = np.vstack((slip_vector_x, slip_vector_y)).T
    for i in range(len(elements)):
        slip_vector_s[i], slip_vector_n[i] = (
            slip_vector[i, :] @ elements[i]["rotation_matrix"]
        )

# plot geometry of mesh
plt.figure(figsize=(15, 2))
bemcs.plot_element_geometry(elements)

# plot slip function
plt.figure(figsize=(15, 3))
plt.subplot(1, 2, 1)
plt.plot((x1 + x2) / 2, slip_vector_s, "o-", label="$s_{shear}$")
plt.plot((x1 + x2) / 2, slip_vector_n, "o-", label="$s_{tensile}$")
plt.xlabel("x")
plt.ylabel("Imposed Slip")
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot((x1 + x2) / 2, slip_vector_x, "o-", label="$s_x$")
plt.plot((x1 + x2) / 2, slip_vector_y, "o-", label="$s_y$")
plt.xlabel("x")
plt.ylabel("Imposed Slip")
plt.grid()
plt.legend()
plt.show()

# design matrices (in x,y coordinates) for slip and slip gradients at each 3qn
matrix_slip, matrix_slipgradient = bemcs.get_designmatrix_xy_3qn(elements)

matrix_system = np.zeros((len(elements) * 6, len(elements) * 6))

bc_vector = np.zeros((len(elements) * 6, 1))
# slip OR slip gradients at the boundaries (we set them to 0 here)
bc_vector[0:2] = 0.0
bc_vector[-2:] = 0.0

bc_vector[2:-3:6] = slip_vector_x.reshape(-1, 1)
bc_vector[3:-2:6] = slip_vector_y.reshape(-1, 1)

# Linear operator for BCs as a matrix
# boundary nodes
# matrix_system[0:2,:] = matrix_slipgradient[0:2:]
# matrix_system[-2:,:] = matrix_slipgradient[-2:,:]
matrix_system[0:2, :] = matrix_slip[0:2:]
matrix_system[-2:, :] = matrix_slip[-2:, :]

# patch center nodes
matrix_system[2:-3:6, :] = matrix_slip[2:-3:6, :]
matrix_system[3:-2:6, :] = matrix_slip[3:-2:6, :]

# overlapping interior nodes (apply slip continuity and smoothness)
# continuity
matrix_system[4:-7:6, :] = (
    matrix_slip[4:-7:6, :] - matrix_slip[6:-5:6, :]
)  # x component
matrix_system[5:-6:6, :] = (
    matrix_slip[5:-6:6, :] - matrix_slip[7:-4:6, :]
)  # y component
# smoothness
matrix_system[6:-5:6, :] = (
    matrix_slipgradient[4:-7:6, :] - matrix_slipgradient[6:-5:6, :]
)  # x component
matrix_system[7:-4:6, :] = (
    matrix_slipgradient[5:-6:6, :] - matrix_slipgradient[7:-4:6, :]
)  # y component

# compute quadratic node coefficients (in local (s,n) coordinates)
quadratic_coefs = np.linalg.inv(matrix_system) @ bc_vector

# extract (s,n) components and store them in 2 separate vectors
quadratic_coefs_s = np.zeros((len(elements) * 3, 1))
quadratic_coefs_n = np.zeros((len(elements) * 3, 1))
for i in range(len(elements)):
    quadratic_coefs_s[3 * i : 3 * (i + 1)] = quadratic_coefs[6 * i : 6 * i + 3]
    quadratic_coefs_n[3 * i : 3 * (i + 1)] = quadratic_coefs[6 * i + 3 : 6 * (i + 1)]

### Plot displacement, stresses and slip

In [ ]:
# compute displacement and stress kernels
# Elastic parameters
mu = 1
nu = 0.25
n_obs = 100
width = 1.5
x_obs = np.linspace(-width, width, n_obs)
y_obs = np.linspace(-width / 2, width / 2, n_obs)

x_obs, y_obs = np.meshgrid(x_obs, y_obs)
x_obs = x_obs.flatten()
y_obs = y_obs.flatten()

# compute shear and tensile kernels
# kernels are [Kxx,Kyy,Kxy,Gx,Gy] x [Nobs x Ncoefficients]
# the last argument passed is a flag for shear/tensile source (1 - shear, 0 - tensile)
kernels_s = bemcs.get_quadratic_displacement_stress_kernel(
    x_obs, y_obs, elements, mu, nu, 1
)
kernels_n = bemcs.get_quadratic_displacement_stress_kernel(
    x_obs, y_obs, elements, mu, nu, 0
)

# Plot displacements and stresses
ux = kernels_s[3] @ quadratic_coefs_s + kernels_n[3] @ quadratic_coefs_n
uy = kernels_s[4] @ quadratic_coefs_s + kernels_n[4] @ quadratic_coefs_n
sxx = kernels_s[0] @ quadratic_coefs_s + kernels_n[0] @ quadratic_coefs_n
syy = kernels_s[1] @ quadratic_coefs_s + kernels_n[1] @ quadratic_coefs_n
sxy = kernels_s[2] @ quadratic_coefs_s + kernels_n[2] @ quadratic_coefs_n
n_skip_plot = 21
bemcs.plot_displacements_stresses(
    elements, n_obs, ux, uy, sxx, syy, sxy, x_obs, y_obs, n_skip_plot
)

# Extract slip at nodes from quadratic coefficients
slip_nodes = matrix_slip @ quadratic_coefs
slip_nodes_x = slip_nodes[0::2]
slip_nodes_y = slip_nodes[1::2]
slip_vector = np.hstack((slip_nodes_x, slip_nodes_y))
# rotate from (x,y) to (s,n) for slip vector
slip_nodes_s = np.zeros((3 * len(elements), 1))
slip_nodes_n = np.zeros((3 * len(elements), 1))

for i in range(len(elements)):
    dummy = slip_vector[3 * i : 3 * (i + 1), :] @ elements[i]["rotation_matrix"]
    slip_nodes_s[3 * i : 3 * (i + 1), 0] = dummy[:, 0]
    slip_nodes_n[3 * i : 3 * (i + 1), 0] = dummy[:, 1]

xnodes = np.zeros((3 * len(elements), 1))
xnodes[0::3, 0] = x1
xnodes[2::3, 0] = x2
xnodes[1::3, 0] = 0.5 * (x1 + x2)

plt.figure(figsize=(15, 3))
plt.subplot(1, 2, 1)
plt.plot(xnodes, slip_nodes_x, "b.-", label="$s_x$")
plt.plot(xnodes, slip_nodes_y, "g.-", label="$s_y$")
plt.plot(xnodes[1:-1:3, 0], slip_nodes_x[1:-1:3, 0], "bo", label="$s_x$ at centers")
plt.plot(xnodes[1:-1:3, 0], slip_nodes_y[1:-1:3, 0], "go", label="$s_y$ at centers")
plt.xlabel("x")
plt.ylabel("Slip at nodes")
plt.legend()
plt.grid()
plt.subplot(1, 2, 2)
plt.plot(xnodes, slip_nodes_s, "k.-", label="$s_s$")
plt.plot(xnodes, slip_nodes_n, "r.-", label="$s_n$")
plt.plot(xnodes[1:-1:3, 0], slip_nodes_s[1:-1:3, 0], "ko", label="$s_s$ at centers")
plt.plot(xnodes[1:-1:3, 0], slip_nodes_n[1:-1:3, 0], "ro", label="$s_n$ at centers")
plt.xlabel("x")
plt.ylabel("Slip at nodes")
plt.legend()
plt.grid()
plt.show()